# Introduction

It may be necessary to modify a skeleton from that created in with a Config. This could include the changing of parts, the application of default parameters, and/or the expansion of datastructures.

First off, the skeleton that was created in example_1 can be loaded.

In [ ]:
import json
from pathlib import Path
import copy
import pprint
from bom_analysis.base import BaseConfig as Config

Config.define_config(config_path="./files/example_config.json")

with open(Path("example_Creating_a_Skeleton_from_Settings.json"), "r") as f:
    loaded_skeleton = json.load(f)
pprint.pprint(loaded_skeleton)

## Modifying a Skeleton

Now that the skeleton is loaded it can be edited. This can be done by passing it a settings dictionary to the SettingsParser. 

The best way to look at the SettingsParser is that it has found the bones of a skeleton laid out in the ground and is now rearranging them, adding new bones, and adding new features to the bones.

In [ ]:
from bom_analysis.parsers import SettingsParser

SettingsParser can be given a number of special keys as strings - the input of which can be checked. These keys are:
* "part_changes" - the swapping out of parts in the skeleton my modifying the children
* "other_changes" - non-structural changes to the skeleton
* "top" - as in config, (the value of which will be defaulted to) top defines the top of the heirarchy
* "parts" - as in config, allows the supply of new .jsons containing parts for the skeleton
* "modules" - the assessment modules a skeleton will undergo. In this example, the running of these modules will not be shown but they can be used to add features to the skeleton
* "materials" - the materials which will make up the componet. It is necessary to specify as a check is run to see if material exists


### Modifying Top
The top of the hierarchy can be changed but to do so the parts must be supplied as input.

This allows the skeleton top to be restructured. It is more likely that instead of changing the top level component, other component changes will be made. 

"part_changes" and "other_changes" just work by altering the dictionary and, therefore, should be set like this in the settings.

### Making Part Changes

Making part changes can happen within the code by supplying the required information to the parser, or following initialisation and modification of the parser. This is switched by the operate optional input.

*operate*
> bolean whether rebuild the skeleton with the part changes

In [ ]:
skeleton = copy.deepcopy(loaded_skeleton)

settings = {
    "top": {"ref": "blanket"},
    "part_changes": {
        "breeding_zone": {"children": {"stiffener": {"type": "beer_box"}}}
    },
}

parser = SettingsParser(settings, skeleton, operate=False)

If a part change is made the entire component will be rebuilt. This is to ensure that there are not parameters/storage left over. In this example, as the blanket requires rebuilding, the part information will be updated within the parser

In [ ]:
parts = {
    "assembly": {
        "class_str": ["bom_analysis.bom.Assembly"],
        "params_name": ["assembly"],
    },
    "component": {
        "class_str": ["bom_analysis.bom.Component"],
        "params_name": ["component"],
    },
    "blanket": {
        "inherits": ["assembly"],
        "children": {
            "breeding_zone": {"type": "breeding_pins"},
            "manifold": {"type": "double_wall_mf"},
        },
    },
    "breeding_pins": {"inherits": ["component"]},
    "double_wall_mf": {"inherits": ["component"]},
}

parameters = {
    "component": {"mass": {"var": "mass", "unit": "kg", "value": None}},
    "assembly": {
        "number_of_components": {
            "var": "number_of_components",
            "unit": None,
            "value": None,
        }
    },
}
parser.parameters.update(parameters)
parser.vertebrae.update(parts)

At this stage, SettingsParser has no idea what beer_box is, therefore it has rebuilt the skeleton. This information can be supplied within settings by settings["parts"]["location"] (defaults when from_file=True) or passed to the functions as a dict.

In [ ]:
new_components = {
    "beer_box": {
        "inherits": ["component"],
        "shape": "square",
        "params_name": ["box_structure"],
    }
}

new_parameters = {
    "box_structure": {
        "NEW": {
            "descr": None,
            "name": "NEWparam",
            "source": "Input",
            "unit": "N/A",
            "value": None,
            "var": "NEW",
        }
    }
}
parser.vertebrae.update(new_components)
parser.parameters.update(new_parameters)

Now that the information has been added to the parser, the skeleton can undergo the surgery.

In [ ]:
parser.surgery()
pprint.pprint(parser.skeleton)

This shows how the skeleton can be modified to allow children to change. As in the previous example, the special strings ("params_name", "inherits"...) can be expanded on. The bones can then be expanded on by adding marrow.

### Adding Storage
#### Other Storage Types
The examples have already used the parameters storage types but the BOM analysis has a few others provided (external ones can be used, the "class_str" just requires definition.
* DFClass - a wrapper for a pandas DataFrame
* MaterialData - a wrapper for material properties (DataFrame input and CoolProps already wrapped)

These can be specified by supplying adding to the skeleton. All storage types will be searched in supplied dictionaries when "inherits" is specified.

In [ ]:
parser.vertebrae["double_wall_mf"]["key_coords"] = {"inherits": ["coordinates"]}
print("\n\n---Before Marrow Added---\n")
pprint.pprint(parser.skeleton["manifold"])

storage = {
    "coordinates": {
        "class_str": ["bom_analysis.base.DFClass"],
        "desc": "a database of coordinates",
    }
}

print("\n\n---After Marrow Added---\n")
parser.storage.update(storage)
parser.surgery()
pprint.pprint(parser.skeleton["manifold"])

The inheritance for storage works in exactly the same way as children, therefore, there is the ability to stack/inherit from multiple dictionaries along a long chain.

#### Modules
Modules can be given requirements which means that particular required Storage Class/parameter/keys can be specified. In the example, say an FEA is required on the components, but storage may be DataFrame required for the nodes.

To mesh the requirements was for a node dataframe to be specified which has been automatically populated in the skeleton which the information being based to add_marrow - as with the above, this can happen automattically by supplying a location of the .json to the config.

In [ ]:
module_definition = {
    "meshing": {
        "args": ["breeding_zone", "manifold"],
        "run": "mesh",
        "requirements": {
            "breeding_zone": {"nodes": {"class_str": ["pandas.DataFrame"]}},
            "manifold": {"nodes": {"class_str": ["pandas.DataFrame"]}},
        },
    }
}
modules_to_run = {"order": {"0": "meshing"}}

parser.update_settings({"modules": modules_to_run})

parser.modules.update(module_definition)
print(parser)

## Adding Defaults

As with everything, default values/materials can be loaded from a file and used to populate the skeleton. All that is required is the nested dictionary to the item which requires editing.

In [ ]:
defaults = {
    "manifold": {
        "params": {"data": {"mass": {"value": 100}}},
        "material": {"name": "steel"},
    }
}
parser.defaults.update(defaults)
parser.surgery()

pprint.pprint(parser.skeleton["manifold"])

Other changes works in exactly the same way but the changes can be specified directly in the settings

## Materials
Materials was mentioned previously as a special input to the skeleton. It is still built like any other storage with the exception that there is a check for the materials in an order of databases contained within the config.

In [ ]:
from bom_analysis.base import BaseConfig as Config
from bom_analysis.utils import Translator

Config.define_config(config_path="./files/example_config.json")
Translator.define_translations(Config.translations)
pprint.pprint(Config.materials)

As with everything convered "class_str" is a special key and contains the class which will define that material. The order defines the priority order for the materials - in the above example all materials will be searched for in CoolProps and if not returned, searched for in a DataFrame material wrapper.

*Note that the "translate_to" string gives the translation used in the transalator covered in example 1.*

Normally the materials are selected within add_marrow as a full build will supply "inherits" to the material so that it can inherit solid/fluid storage.

In [ ]:
parser.update_config()
material = parser.skeleton["manifold"]["material"]
parser.select_library(material)
pprint.pprint(parser.skeleton["manifold"]["material"])

From this it is shown that CoolProps does not have eurofer so the data has been taken from the DataFrame library with the dataframe loaded from "./files/CD-STEP-00824.json"

# Summary
The example given has modified the skeleton by altering the settings.

It has shown the various types of special keys used to rebuild the skeleton, add_defaults, populate materials classes etc.

In [ ]:
import json
from pathlib import Path

with open(Path("./example_Modifying_a_Skeleton_from_Settings.json"), "w") as f:
    json.dump(parser.skeleton, f, indent=4)

In [ ]:
parser.skeleton